<a href="https://colab.research.google.com/github/visiont3lab/my-dashboard/blob/master/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/visiont3lab/my-dashboard.git

Cloning into 'my-dashboard'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 74 (delta 31), reused 32 (delta 12), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [2]:
%cd my-dashboard/


/content/my-dashboard


In [0]:
%%capture
!pip install -r requirements.txt

In [0]:
!pip install flask-ngrok

In [9]:
%%writefile assets/typography.css
body {
    padding: 20px 300px 0px 300px;
    margin : 0px 0px 0px  0px;
    /*padding : 0px 0px 0px 0px;*/
    background-color: red;
}

Overwriting assets/typography.css


In [12]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from flask_ngrok import run_with_ngrok

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

def plot_matplotlib(df,xx_string,yy_string,provincia, style='go--',mylabel="totale_casi"):
    with plt.style.context("dark_background"):
        xx = df[xx_string].values
        yy = df[yy_string].values
        plt.figure(figsize=(18,8))
        plt.plot()
        plt.plot(xx,yy,style,linewidth=2,label=mylabel)
        plt.legend()
        plt.title(provincia + "Analisi Matplotlib")
        plt.xlabel("data")
        plt.ylabel("totale casi")
        plt.savefig("assets/images/matplotlib.png", transparent=False)
        plt.show()

def plot_pandas(df,xx_string,yy_string, provincia):
    with plt.style.context("seaborn"):
        ax = df.set_index(xx_string).plot(y=yy_string,figsize=(18,8),title=provincia +" Analisi Pandas",grid=True)
        ax.figure.savefig("assets/images/pandas-matplotlib.png", transparent=False)
        plt.show()

def plot_plotly(df,xx_string,yy_string,provincia):
    xx = df[xx_string].values.tolist()
    yy = df[yy_string].values.tolist()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
            x = xx,
            y = yy,
            name= "totale_casi",
            mode="lines+markers",
        )
    )
    fig.update_layout(
        title=dict(
            text =provincia + " Analisi Plotly",
            y = 0.9,
            x = 0.5,
            xanchor = "center",
            yanchor = "top",
        ),
        legend=dict(
            y = 0.9,
            x = 0.03,
        ),
        xaxis_title="data",
        yaxis_title="totale casi",
        hovermode='x',  #['x', 'y', 'closest', False]
        plot_bgcolor = "rgb(10,10,10)",
        paper_bgcolor="rgb(0,0,0)"
    )
    return fig
    #fig.write_image("assets/images/plotly.png")

def plot_regione(df,regione):
    df_filt_reg = df[df["denominazione_regione"]==regione]

    fig = go.Figure()
    province = list(df_filt_reg["denominazione_provincia"].unique())
    province.remove("In fase di definizione/aggiornamento")
    for provincia in province:
        df_filt_prov = df_filt_reg[df_filt_reg["denominazione_provincia"]==provincia]
        xx = df_filt_prov["data"]
        yy = df_filt_prov["totale_casi"].values
        fig.add_trace(go.Scatter(x = xx,y = yy,name=provincia + ": totale casi" ,mode="lines+markers"))
        fig.update_layout(title=regione, hovermode="x",plot_bgcolor = "rgb(10,10,10)",
        paper_bgcolor="rgb(0,0,0)")
    return fig

df = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv")
df["data"] = pd.to_datetime(df["data"]).dt.date
provincia = "Ravenna"
df_filt = df[df["denominazione_provincia"]==provincia]

#plot_pandas(df_filt,"data","totale_casi", provincia)
#plot_matplotlib(df_filt,"data","totale_casi", provincia)
fig = plot_plotly(df_filt,"data","totale_casi", provincia)
fig_reg = plot_regione(df,"Emilia-Romagna")
#fig.show()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
run_with_ngrok(server) 

app.layout = html.Div([
        html.Div(
        [   
            dcc.Markdown('''
                # Analisi Covid 19 (nuovo update) Ciao  fdsfdsfdsfsa asd
                Andremo a eseguire un esercizio sul dataset covid.
                
                ## Esercizio n.1 
                >  Plottare l'andamento nel tempo dei contagiati della propria provincia.
                '''),  
        ]),
        html.Div(
        [   
            dcc.Graph(figure=fig),
        ]),
        html.Div(
        [   
            dcc.Markdown('''
                ## Esercizio n.2
                >  Plottare l'andamento nel tempo dei contagiati di tutte le pronvice dell'emilia romagna.
                '''),  
        ]),
        html.Div(
        [   
            dcc.Graph(figure=fig_reg),
        ]),
    ],style={"margin": "0", "padding": "0"})

if __name__ == '__main__':
    server.run()
    #app.run_server(host="0.0.0.0") #,debug=True,port=8900) #host="0.0.0.0", port=8900)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b38bcdd9dba6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Jun/2020 17:49:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:50] "GET /assets/typography.css?m=1591638495.9949372 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:50] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_2m1591638339.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:50] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_2m1591638339.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:50] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_2m1591638339.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:50] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_2m1591638339.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:50] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_8_1m1591638341.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 17:49:51] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_8_1m1591638341.js HTTP/1

In [13]:
!python app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://26754b7a6fe5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/matplotlib/_pylab_helpers.py", line 76, in destroy_all
    gc.collect(1)
KeyboardInterrupt


In [15]:
!git status


On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   app.py
	modified:   assets/typography.css

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git add app.py assets/typography.css

In [0]:
!git config --global user.email "visiont3lab@gmail.com"

In [0]:
!git config --global user.name "visiont3lab"

In [20]:
!git commit -m "Change colab"

[master d8fc484] Change colabÇ
 2 files changed, 8 insertions(+), 8 deletions(-)


In [21]:
!git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address
